<a href="https://colab.research.google.com/github/AndyBaiMQC/rl-project-20/blob/master/Car_Racing_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri May  1 07:05:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Reinforcement Learning (COMP-767) Project

---

[CarRacing-v0](https://gym.openai.com/envs/CarRacing-v0/)

---



In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Udem/reinf')

In [0]:
import sys
sys.path.append('/content')

In [2]:
!nvidia-smi

# dependencies to make gym work on colab
!pip install -q box2d-py pyglet
!pip install -q gym[Box_2D]
!apt-get install x11-utils

!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()
# https://stackoverflow.com/questions/50107530/how-to-render-openai-gym-in-google-colab

Fri May  1 11:01:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import gym
import tensorflow as tf
import tensorflow.nn as nn
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense
import tensorflow.keras.backend as K
import tensorflow_probability as tfp
from tensorflow.keras import optimizers
from torch.distributions import Beta
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
from tqdm.notebook import tqdm

In [0]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.
tf.keras.backend.set_floatx('float64')

In [0]:
# hyperparams
gamma = 0.99
# repeats an action n times for each step
action_repeat = 2 
# channel dimension for input to CNN model. 4 is the number of channels in the image
img_stack = 4
seed = 0
render = False
# logging, ignore
log_interval = 10
# datatype signature to store a list object
transition = np.dtype([('s', np.float64, (96, 96, img_stack)), ('a', np.float64, (3,)), ('a_logp', np.float64),
                       ('r', np.float64), ('s_', np.float64, (96, 96, img_stack))])

In [0]:
import gym
import numpy as np

class Env():
    """
    Environment wrapper for CarRacing 
    """

    def __init__(self,directory="vid_save_dir"):
        seed = 0
        self.env = gym.make('CarRacing-v0')
        self.env.seed(seed)
        self.reward_threshold = self.env.spec.reward_threshold
        # self.count = 0

    def reset(self):
        self.counter = 0
        self.av_r = self.reward_memory()

        self.die = False
        img_rgb = self.env.reset()
        img_gray = self.rgb2gray(img_rgb)
        self.stack = [img_gray] * img_stack  # four frames for decision
        # self.monitor_env = gym.wrappers.Monitor(self.env, vid_save_dir, video_callable=lambda x: x%10 == 0)

        return np.array(self.stack)

    # takes a step, returns stack, reward, done, die
    # done = True: episode ended forcefully due to car diverging in all green zone
    # die = True: episode ended via env
    def step(self, action):
        total_reward = 0
        for i in range(action_repeat):
            img_rgb, reward, die, _ = self.env.step(action)
            # don't penalize "die state"
            if die:
                reward += 100
            # green penalty
            if np.mean(img_rgb[:, :, 1]) > 185.0:
                reward -= 0.05
            total_reward += reward
            # if no reward recently, end the episode
            done = True if self.av_r(reward) <= -0.1 else False
            if done or die:
                break
        img_gray = self.rgb2gray(img_rgb)
        self.stack.pop(0)
        self.stack.append(img_gray)
        assert len(self.stack) == img_stack
        return np.array(self.stack), total_reward, done, die

    def render(self, *arg):
        self.env.render(*arg)

    @staticmethod
    def rgb2gray(rgb, norm=True):
        # rgb image -> gray [0, 1]
        gray = np.dot(rgb[..., :], [0.299, 0.587, 0.114])
        if norm:
            # normalize
            gray = gray / 128. - 1.
        return gray

    # maintaisn reward history buffer
    @staticmethod
    def reward_memory():
        # record reward for last 100 steps
        count = 0
        length = 100
        history = np.zeros(length)

        def memory(reward):
            nonlocal count
            history[count] = reward
            count = (count + 1) % length
            return np.mean(history)
        return memory

In [0]:
class ActorCriticModel(tf.keras.Model):
    """
    Actor-Critic Network for PPO
    """

    def __init__(self):
        super(ActorCriticModel, self).__init__()
        self.cnn_base = tf.keras.Sequential() # # input shape (4, 96, 96)
        self.cnn_base.add(Conv2D(8,kernel_size=4, strides=2, activation='relu', input_shape=(96, 96, 4),)) # data_format='channels_first'))
        self.cnn_base.add(Conv2D(16,kernel_size=3, strides=2, activation='relu',)) #data_format='channels_first')) # (8, 47, 47) 
        self.cnn_base.add(Conv2D(32,kernel_size=3, strides=2, activation='relu',)) # data_format='channels_first')) # (16, 23, 23)
        self.cnn_base.add(Conv2D(64,kernel_size=3, strides=2, activation='relu',)) # data_format='channels_first')) # (32, 11, 11)
        self.cnn_base.add(Conv2D(128,kernel_size=3, strides=1, activation='relu',)) # data_format='channels_first')) # (64, 5, 5)
        self.cnn_base.add(Conv2D(256,kernel_size=3, strides=1, activation='relu',)) # data_format='channels_first')) # (128, 3, 3)
        # output shape (256, 1, 1)

        # print(self.cnn_base.summary())
        self.v = tf.keras.Sequential([ # input shape 256
          Dense(100,activation='relu',input_shape=(256,)),
          Dense(1)
        ])
        self.fc = tf.keras.Sequential([ # input shape 256
          Dense(100,input_shape=(256,))
        ])
        self.alpha_head = tf.keras.Sequential([ # input shape 100
          Dense(3, activation='softplus', input_shape=(100,))
        ])
        self.beta_head = tf.keras.Sequential([ #input shape 100
          Dense(3, activation='softplus', input_shape=(100,)),
        ])
        # self.apply(self._weights_init)

    # @staticmethod
    # def _weights_init(m):
    #     if isinstance(m, nn.Conv2d):
    #         nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
    #         nn.init.constant_(m.bias, 0.1)

    def call(self, x):
        x = self.cnn_base(x)

        x = tf.reshape(x,[-1,256])

        v = self.v(x)
        x = self.fc(x)
        alpha = self.alpha_head(x) + 1
        beta = self.beta_head(x) + 1
        return (alpha, beta), v

In [0]:
# Learning Agent that trains the environment
class Agent():
    """
    Agent for training
    """
    def __init__(self,max_grad_norm=0.5,clip_param=0.1,ppo_epoch=10,buffer_capacity=2000,batch_size=128):
        self.max_grad_norm = max_grad_norm
        self.clip_param = clip_param  # epsilon in clipped loss
        self.ppo_epoch = ppo_epoch
        self.buffer_capacity, self.batch_size = buffer_capacity, batch_size # 2000, 128

        self.training_step = 0
        self.net = ActorCriticModel()
        self.buffer = np.empty(self.buffer_capacity, dtype=transition)
        self.counter = 0

        self.optimizer = optimizers.Adam(lr=1e-3)
        self.loss = tf.keras.losses.Huber()
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')

    def select_action(self, state):
        # state = torch.from_numpy(state).double().to(device).unsqueeze(0)
        # with torch.no_grad():
        alpha, beta = self.net(state)[0]
        # samples action from Beta distribution
        dist = tfp.distributions.Beta(alpha, beta)
        action = dist.sample()
        a_logp = dist.log_prob(action)
        a_logp = tf.reduce_sum(a_logp, axis=1)

        action = tf.squeeze(action).numpy()
        a_logp = a_logp.numpy()
        return action, a_logp

    def save_param(self):
        torch.save(self.net.state_dict(), 'param/ppo_net_params.pkl')

    def store(self, transition):
        self.buffer[self.counter] = transition
        self.counter += 1
        if self.counter == self.buffer_capacity:
            self.counter = 0
            return True
        else:
            return False

    # update function
    def update2(self,):
        self.training_step += 1

        # state, action, reward, next_state
        s = tf.constant(self.buffer['s'], dtype=tf.float64)
        a = tf.constant(self.buffer['a'], dtype=tf.float64)
        r = tf.constant(self.buffer['r'], dtype=tf.float64)
        r = tf.reshape(r,(-1, 1))
        s_ = tf.constant(self.buffer['s_'], dtype=tf.float64)

        old_a_logp = tf.constant(self.buffer['a_logp'], dtype=tf.float64)
        
        target_v = r + gamma * self.net(s_)[1]
        adv = target_v - self.net(s)[1]

        dataset = tf.data.Dataset.from_tensor_slices({"s": s, "a": a, "old_a_logp": old_a_logp,
                                                      "target_v":target_v, "adv":adv}).batch(self.batch_size).as_numpy_iterator()
        dataset = list(dataset)
        for _ in range(self.ppo_epoch):
            for data in dataset:
                s = data["s"]
                a = data["a"]
                old_a_logp = data["old_a_logp"]
                target_v = data["target_v"]
                adv = data["adv"]
                self.train_step2(s, a , old_a_logp, adv, target_v)

    # train step given s, a, old_a_logp, adv, target_v
    def train_step2(self,s, a, old_a_logp, adv, target_v):
        with tf.GradientTape() as tape:
            # training=True is only needed if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            s = tf.constant(s,dtype=tf.float64)
            a = tf.constant(a,dtype=tf.float64)
            old_a_logp = tf.constant(old_a_logp,dtype=tf.float64)
            adv = tf.constant(adv,dtype=tf.float64)
            target_v = tf.constant(target_v,dtype=tf.float64)

            (alpha, beta), v = self.net(s)
            dist = tfp.distributions.Beta(alpha, beta)
            # a_logp = dist.log_prob(a).sum(axis=1, keepdim=True)
            a_logp = dist.log_prob(a)
            a_logp = tf.reduce_sum(a_logp, axis=1, keepdims=True)
            
            ratio = tf.math.exp(a_logp - old_a_logp)
            surr1 = ratio * adv
            surr2 = tf.clip_by_value(ratio, 1.0 - self.clip_param, 1.0 + self.clip_param) * adv
            action_loss = tf.reduce_mean(-tf.math.minimum(surr1, surr2))
            v = tf.squeeze(v)
            target_v = tf.squeeze(target_v)
            value_loss = self.loss(v, target_v)
            loss = action_loss + 2. * value_loss
        gradients = tape.gradient(loss, self.net.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.net.trainable_variables))
        self.train_loss(loss)

In [10]:
agent = Agent(buffer_capacity=2000)
env = Env()
# if args.vis:
#     draw_reward = DrawLine(env="car", title="PPO", xlabel="Episode", ylabel="Moving averaged episode reward")

training_records = []
running_score = 0
state = env.reset()
losses = []
running_scores = []
scores = []
# epochs
for i_ep in range(1100):
    score = 0
    state = env.reset()
    state = np.moveaxis(state, 0, -1)
    state = np.expand_dims(state, axis=0).astype(np.float64)
    # no. of steps
    for t in range(1000):
        action, a_logp = agent.select_action(state)
        state_, reward, done, die = env.step(action * np.array([2., 1., 1.]) + np.array([-1., 0., 0.]))
        state_ = np.moveaxis(state_, 0, -1)
        state_ = np.expand_dims(state_, axis=0).astype(np.float64)
        if render:
            env.render()
        if agent.store((state, action, a_logp, reward, state_)):
            print('updating')
            agent.update2()
            loss = agent.train_loss.result()
            losses.append(loss)
            print('Loss: {:.5f}'.format(loss))
        score += reward
        state = state_
        if done or die:
            break
    running_score = running_score * 0.99 + score * 0.01
    scores.append(score)
    running_scores.append(running_score)
    if i_ep % log_interval == 0 or True:
        print('Ep {}\tLast score: {:.2f}\tMoving average score: {:.2f}\t'.format(i_ep, score, running_score))
    if running_score > env.reward_threshold:
        print("Solved! Running reward is now {} and the last episode runs to {}!".format(running_score, score))

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1143..1442 -> 299-tiles track
Track generation: 1087..1369 -> 282-tiles track
episode-end reward: -0.09999999999999964
Ep 0	Last score: 35.41	Moving average score: 0.35	
Track generation: 964..1212 -> 248-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1176..1474 -> 298-tiles track
Ep 1	Last score: -19.49	Moving average score: 0.16	
Track generation: 1283..1608 -> 325-tiles track
updating
Loss: 0.43352
episode-end reward: -0.10000000000000142
Ep 2	Last score: 17.90	Moving average score: 0.33	
Track generation: 1217..1526 -> 309-tiles track
Ep 3	Last score: -20.42	Moving average score: 0.13	
Track generation: 1096..1374 -> 278-tiles track
episode-end reward: -0.10000000000000142
Ep 4	Last score: 29.42	Moving average score: 0.42	
Track generation: 1198..1501 -> 303-tiles track
updating
Loss: 0.40219
Ep 5	Last score: -29.27	Moving average score: 0.12	
Track generation: 1159..1453 -> 294-tiles track
episode-end reward:

KeyboardInterrupt: ignored

In [0]:
env.close()

In [14]:
agent.net.save('first_model',save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: first_model/assets


In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
class Test():
    def __init__(self,model_path):
        self.model_path = model_path
        self.net = tf.keras.models.load_model(self.model_path)
        self.env = Env()

    def reload_model(self,model_path):
        self.model_path = model_path
        self.net = tf.keras.models.load_model(self.model_path)

    def select_action(self, state):
        # state = torch.from_numpy(state).double().to(device).unsqueeze(0)
        # with torch.no_grad():
        alpha, beta = self.net(state)[0]
        dist = tfp.distributions.Beta(alpha, beta)
        action = dist.sample()
        a_logp = dist.log_prob(action)
        a_logp = tf.reduce_sum(a_logp, axis=1)

        action = tf.squeeze(action).numpy()
        a_logp = a_logp.numpy()
        return action, a_logp

    def run_test(episodes=10):
        env = Env()
        # if args.vis:
        #     draw_reward = DrawLine(env="car", title="PPO", xlabel="Episode", ylabel="Moving averaged episode reward")

        running_score = 0
        state = env.reset()
        losses = []
        running_scores = []
        scores = []
        for i_ep in range(episodes):
            score = 0
            state = env.reset()
            state = np.moveaxis(state, 0, -1)
            state = np.expand_dims(state, axis=0).astype(np.float64)
            for t in range(1000):
                # print(state.shape)
                action, a_logp = self.select_action(state)
                state_, reward, done, die = env.step(action * np.array([2., 1., 1.]) + np.array([-1., 0., 0.]))
                state_ = np.moveaxis(state_, 0, -1)
                state_ = np.expand_dims(state_, axis=0).astype(np.float64)
                if render:
                    env.render()
                score += reward
                state = state_
                if done or die:
                    break
            running_score = running_score * 0.95 + score * 0.05
            scores.append(score)
            running_scores.append(running_score)
            if i_ep % log_interval == 0 or True:
                # if args.vis:
                #     draw_reward(xdata=i_ep, ydata=running_score)
                print('Ep {}\tLast score: {:.2f}\tMoving average score: {:.2f}\t'.format(i_ep, score, running_score))
                # agent.save_param()
            if running_score > env.reward_threshold:
                print("Solved! Running reward is now {} and the last episode runs to {}!".format(running_score, score))
        return

In [39]:
%autoreload 2
from utils import Test
test = Test("/content/first_model")

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [40]:
test.run_test()

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1143..1442 -> 299-tiles track
Track generation: 1087..1369 -> 282-tiles track
Ep 0	Last score: 92.53	Moving average score: 0.93	
Track generation: 964..1212 -> 248-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1176..1474 -> 298-tiles track
Ep 1	Last score: -22.79	Moving average score: 0.69	


Error: ignored